In [1]:

import os
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import torch
import tensorflow as tf

from torch.autograd import grad
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ExponentialLR

from pdefind import *
from PDE_Equation import pde_matrix_mul, sparse_coeff, normalized_xi_threshold, pde_Recover

In [2]:
# Prepare dataset
data = sio.loadmat(os.path.join(os.getcwd(), "data", "Advection_diffusion.mat"))
usol = np.real(data['Expression1'])
usol= usol.reshape((51,51,61,4))

x = usol[:,:,:,0]
y = usol[:,:,:,1]
t = usol[:,:,:,2]
u = usol[:,:,:,3]

X = np.transpose((t.flatten(),x.flatten(), y.flatten()))
y = u.reshape((u.size, 1))
print(X.shape, y.shape)
print('X:', X)

print("y.shape", y.shape)

(158661, 3) (158661, 1)
X: [[ 3.  -5.  -5. ]
 [ 3.1 -5.  -5. ]
 [ 3.2 -5.  -5. ]
 ...
 [ 8.8  5.   5. ]
 [ 8.9  5.   5. ]
 [ 9.   5.   5. ]]
y.shape (158661, 1)


In [3]:
# Add noise
noise_level = 0.1
y_noisy = y + noise_level * np.std(y) * np.random.randn(y.size, 1)


In [4]:
#number_of_samples = 2000
#idx = np.random.permutation(y.size)
#X_train = X[idx, :][:number_of_samples]
#y_train = y_noisy[idx, :][:number_of_samples]
#print('X_train', X_train.shape)
#print('y_train', y_train.shape)

idxs = np.random.choice(y.size, 2000, replace=False)

X_train = torch.tensor(X[idxs], dtype=torch.float32, requires_grad=True)
y_train = torch.tensor(y_noisy[idxs], dtype=torch.float32)
print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)
#print("X shape", X.shape)
#print("y shape", y.shape)
#print('X_train', X_train)

X_train shape torch.Size([2000, 3])
y_train shape torch.Size([2000, 1])


In [5]:
# Setup Network
net = PINN(sizes=[3,20,15,10,5,1], activation=torch.nn.Tanh())
print(net)

PINN(
  (net): Sequential(
    (0): Linear(in_features=3, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=15, bias=True)
    (3): Tanh()
    (4): Linear(in_features=15, out_features=10, bias=True)
    (5): Tanh()
    (6): Linear(in_features=10, out_features=5, bias=True)
    (7): Tanh()
    (8): Linear(in_features=5, out_features=1, bias=True)
  )
)


In [6]:
#Construct Library
def build_library_2D(data, uhat, poly_order, deriv_order):
    # build polynomials
    poly = torch.ones_like(uhat)
    
    # concatinate different orders
    for o in np.arange(1, poly_order+1):
        poly_o = poly[:,o-1:o]*uhat
        poly = torch.cat((poly, poly_o), dim=1)
        #print('poly.shape', poly.shape)
        
    # build derivatives
    # returns gradient of uhat w.r.t. data (id0=spatial, id1=temporal)
    du = grad(outputs=uhat, inputs=data, 
              grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
    
    dudt = du[:, 0:1]
    dudx = du[:, 1:2]
    dudy = du[:, 2:3]
    dudu = grad(outputs=dudx, inputs=data, 
              grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
    dudxx = dudu[:, 1:2]
    dudxy = dudu[:, 2:3]
    dudyy = grad(outputs=dudy, inputs=data, 
              grad_outputs=torch.ones_like(dudxx), create_graph=True)[0]
   
    #dudu = torch.cat((torch.ones_like(dudt), du[:,0:1]), dim=1)
    dudu = torch.cat((torch.ones_like(dudx), torch.ones_like(dudx), dudy, dudxx, dudyy[:, 2:3], dudxy), dim=1)

    
    #for o in np.arange(1, deriv_order):
        #du2 = grad(outputs=dudu[:,o:o+1], inputs=data, 
                  #grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
       # dudx = torch.cat((dudx, du2[:,0:1]), dim=1)


    
    # build all possible combinations of poly and dudu vectors
    theta = None
    for i in range(poly.shape[1]):
        #print('i:', i)
        for j in range(dudu.shape[1]):
            #print('j:', j)
            comb = poly[:,i:i+1] * dudu[:,j:j+1]
            
            if theta is None:
                theta = comb
            else:
                theta = torch.cat((theta, comb), dim=1)
                
    return dudt, theta

In [7]:
uhat = net(X_train)
dudt, theta = build_library_2D(X_train, uhat, poly_order=1, deriv_order=2)
#print(dudu.shape)
print('theta:', theta.shape)
#print('dudu.shape:', dudu.shape)

theta: torch.Size([2000, 12])


In [8]:
polynm = ['1', 'u']
spa_der = ['1', 'u_{x}', 'u_{y}','u_{xx}', 'u_{yy}','u_{xy}']
library_coeffs = pde_matrix_mul(polynm, spa_der)
print('library_coeffs:', library_coeffs)

tot_items = len(library_coeffs)
print('tot_items:', tot_items)

epochs = 43000
#xi = nn.Parameter(torch.randn((1, 1), requires_grad=True, device="cpu", dtype=torch.float32))
xi = nn.Parameter(torch.randn((tot_items, 1), requires_grad=True, device="cpu", dtype=torch.float32))
#xi = torch.tensor([[0.1], [-1]])
#print(xi)
#params = [{'params': net.parameters(), 'lr': 1e-3}]
params = [{'params': net.parameters(), 'lr': 3e-3}, {'params': xi, 'lr': 3e-2}]

optimizer = Adam(params)
scheduler = ExponentialLR(optimizer, .9998)

library_coeffs: ['1', 'u_{x}', 'u_{y}', 'u_{xx}', 'u_{yy}', 'u_{xy}', 'u', 'uu_{x}', 'uu_{y}', 'uu_{xx}', 'uu_{yy}', 'uu_{xy}']
tot_items: 12


In [9]:
def model_identification(features, label, mask, poly_order, deriv_order):
    lamb   = 0
    tolerance = 1e-6
    mask = torch.ones(tot_items, 1)
    print('xi', xi)
    print('mask:', mask.shape)
    lambd  = 1e-5

    
    for epoch in range(epochs):
        optimizer.zero_grad()
        uhat = net(features)
    
        if epoch == 1000:
            lamb = 1
               
        dudt, theta = build_library_2D(features, uhat, poly_order=1, deriv_order=2)
        #print('dudt:', dudt.shape)
        dudt_norm = torch.norm(dudt, dim=0)
        #print('dudt_norm:', dudt_norm.shape)
    
    
        theta_scaling = (torch.norm(theta, dim=0))
        #print('theta_scaling:', theta_scaling.shape)
        #Returns a new tensor with a dimension of size one inserted at the specified position. from 9 it will be 9,1
        theta_norm = torch.unsqueeze(theta_scaling, dim = 1) 
        #print('theta_norm:', theta_norm.shape)
        xi_normalized = xi * (theta_norm / dudt_norm) 
        L1 = lambd * torch.sum(torch.abs(xi_normalized[1:, :]))
        
        l_u   = nn.MSELoss()(uhat, label)
        #l_reg = lamb * torch.mean((dudt - theta @ xi)**2)
        l_reg = torch.mean((dudt - theta @ xi)**2)

        loss = l_u + l_reg + L1
        #print('loss', loss)
    

        
        gradient_loss = torch.max(torch.abs(grad(outputs=loss, inputs=xi, 
              grad_outputs=torch.ones_like(loss), create_graph=True)[0]) / (theta_norm / dudt_norm))
        


        loss.backward(retain_graph=True)
        optimizer.step()
    
        #print("epoch {}/{}, loss={:.10f}".format(epoch+1, epochs, loss.item()), end="\r")
        
        if epoch % 1000 == 0:
            print('loss:', epoch, loss)
            if gradient_loss < tolerance:
                print('Optimizer converged.')
                break
 
    #print('xi_normalized:', xi_normalized)
    xi_list = sparse_coeff(mask, xi.detach().numpy())
    xi_normalized = sparse_coeff(mask, xi_normalized.detach().numpy())
    print('xi_normalized:', xi_normalized)
    
    sparsity = normalized_xi_threshold( xi_normalized, mode='auto')
    print('sparsity:', sparsity)
    
  
    xi_thresholded = np.expand_dims(xi_list[sparsity], axis=1) 
    print('xi_thresholded:', xi_thresholded)
    # Printing current sparse vector 
    print('Coefficient xi:')
    xi_updated = sparse_coeff(sparsity, xi_thresholded)
    print(xi_updated)
    print('Finished')
            
            
    return xi_updated

In [10]:
mask = torch.ones(tot_items, 1)
uhat = net(X_train)
xi_updated= model_identification(X_train, y_train, mask, poly_order=1, deriv_order=2)

xi Parameter containing:
tensor([[-0.7046],
        [ 0.6002],
        [ 0.3122],
        [-1.6245],
        [-0.0065],
        [ 1.8249],
        [ 0.7938],
        [-0.7229],
        [-0.6066],
        [-0.1556],
        [ 0.1365],
        [-2.6641]], requires_grad=True)
mask: torch.Size([12, 1])
loss: 0 tensor(0.8722, grad_fn=<AddBackward0>)
loss: 1000 tensor(0.0043, grad_fn=<AddBackward0>)
loss: 2000 tensor(0.0037, grad_fn=<AddBackward0>)
loss: 3000 tensor(0.0035, grad_fn=<AddBackward0>)
loss: 4000 tensor(0.0033, grad_fn=<AddBackward0>)
loss: 5000 tensor(0.0033, grad_fn=<AddBackward0>)
loss: 6000 tensor(0.0032, grad_fn=<AddBackward0>)
loss: 7000 tensor(0.0032, grad_fn=<AddBackward0>)
loss: 8000 tensor(0.0033, grad_fn=<AddBackward0>)
loss: 9000 tensor(0.0030, grad_fn=<AddBackward0>)
loss: 10000 tensor(0.0030, grad_fn=<AddBackward0>)
loss: 11000 tensor(0.0030, grad_fn=<AddBackward0>)
loss: 12000 tensor(0.0030, grad_fn=<AddBackward0>)
loss: 13000 tensor(0.0029, grad_fn=<AddBackward0>)

In [11]:
print(uhat.shape)
pde_Recover(xi_updated, library_coeffs, equation_form='u_t')

torch.Size([2000, 1])
Burger equation:


'u_t = 0.4451u_{y} + 0.2850u_{xx} + 0.5289u_{yy}'